# **Extração de emoções e Análise de Sentimentos em Letras de músicas do Forró**

In [1]:
import pandas as pd

In [4]:
df = pd.read_csv('./lyrics.csv', sep=';', encoding="utf-8")
#print(f'Quantidade de músicas: { df.id.count() }')
df.head()

,id,title,artist,composer,album,year,lyric
0,1,Não Se Esquecerá,Forrozão Tropykália,Carlinhos.,Emoção Incomparável - Vol. 9,0,Sei que poderá levar um tempo Sei que poderá d...
1,2,Planeta de Cores,Forrozão Tropykália,CARLINHOS GABRIEL.,Emoção Incomparável - Vol. 9,0,Ei Vou deixar como está Vai ser mais uma histó...
2,3,Enquanto Houver Amor,Forrozão Tropykália,Dj Samuca.,Emoção Incomparável - Vol. 9,0,Estou só sem você aqui A saudade me sufoca E m...
3,4,Pronta Pra Te Amar,Forrozão Tropykália,Cristian Lima / Ivo Lima.,Emoção Incomparável - Vol. 9,0,Meu amor porque foi embora Logo agora que eu t...
4,5,Saudade Mata,Forrozão Tropykália,NaN,Emoção Incomparável - Vol. 9,0,Saudade mata Quando agente sente Que o amor d...


# Preprocessing

1. Remoção dos títulos das músicas que contenham os caracteres '()' 

In [5]:
df['title'].replace("\((.*?)\)", '', regex=True, inplace=True)



2. Remoção das músicas duplicadas a partir do título

In [6]:
df['title'] = df['title'].str.lower()
df['title'] = df['title'].str.strip()

titles = [i for i in df['title']]

duplicates = [i for i in titles if titles.count(i) > 1]
print(f'Quantidade de músicas duplicatas: {len(duplicates)}')

Quantidade de músicas duplicatas: 3484


In [9]:
#from google.colab import drive
#drive.mount('/content/drive')

In [10]:
df = df[~df['title'].isin(duplicates)]
print(f'Quantidade de músicas após remoção das duplicatas: {df.id.count()}')

Quantidade de músicas após remoção das duplicatas: 9921


3. Remoção das músicas sem data

In [13]:
df = df[df['year'] > 0]

print(f'Quantidade de músicas com data: {df.id.count()}')

Quantidade de músicas com data: 8176


4. Remoção dos registros sem letras

In [14]:
df = df[~df['lyric'].isnull()]
df['lyric'].count()

8133

## Criando bases de treino e teste

1. Base de Treino

In [15]:
limit = int(df['id'].count() * (80/100)) + 1
train_df = df.head(limit)
train_df.to_csv('./lyrics_train.csv', index=False)

2. Base de Teste

In [16]:
limit = int(df['id'].count() * (20/100))
train_df = df.head(limit)
train_df.to_csv('./lyrics_test.csv', index=False)

In [ ]:
#!pip install -r requirements.txt

### Carregando dataframes

In [ ]:
df= pd.read_csv('./lyrics_train.csv', encoding='utf-8')
#df_test = pd.read_csv('/content/drive/MyDrive/monografia/tcc/projeto/lyrics_test.csv', encoding='utf-8')
"""
Lexicon source is (C) 2016 National Research Council Canada (NRC) 
and library is for research purposes only.  
Source: http://sentiment.nrc.ca/lexicons-for-research/
"""
lexicon = pd.read_csv('./pt-nrc-lexicon.csv', sep=';', encoding='utf-8')

In [ ]:
df['pp_lyric'] = df['lyric'].tail(5).apply(clean_lyrics)

### Funções utilitárias

In [ ]:
#carregando lista de acordes para remoção dentro das letras
chords_df = pd.read_csv('./chords-only.txt')
chords_dicty = chords_df.to_dict()
chords = set()

for i in chords_dicty['chord']:
    chords.add(chords_dicty['chord'][i].strip())
    
#função para remoção de acordes dentro de um texto
def remove_chords(text):
    txt_list = text.split()
    return (' ').join(
            word.strip()
            for word in txt_list
            if word.strip() not in chords
        )    

In [ ]:
import unidecode as und
def to_unidecode(text):
    txt=""
    for j in text.split():
        txt+= und.unidecode(j) + " "
    return txt.strip()

In [ ]:
import string
import re
import spacy

nlp = spacy.load('pt_core_news_sm')

def clean_lyrics(text):
  txt = ''
  #trocando letras especiais por normais
  txt = (' ').join(to_unidecode(word) for word in text.split())
  #removendo acordes
  txt = remove_chords(text)
  #removendo tags de reptição 2x, 4x, etc
  txt = txt.replace('[', '(').replace(']', ')')
  txt = re.sub(r'\([\d]x\)', '', txt)
  #removendo palavras de orientação musical
  txt = re.sub(r'\((.*?)\)', '', txt)  
  #removendo pontuacao
  txt = txt.translate(str.maketrans("", "", string.punctuation))
  #caracteres em minusculo   
  txt = txt.lower()
    
  return txt

In [ ]:
def preprocessing(text):
    #stop_words = spacy.lang.pt.stop_words.STOP_WORDS
    #lematização e remoção de stop words
    #doc = nlp(txt) 
    #txt = (' ').join(token.lemma_ for token in doc 
    #                 if not token.is_stop
    #                 and token.is_alpha)
    pass